In [ ]:
#;.pykx.disableJupyter()

In [ ]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

# Joins

**Learning Outcomes**

To understand: 
+ Simple table joins
+ Common Keyed joins - lj, ij
+ Bitemporal (asof) joins 
+ How to use aj 

# Introduction 

In a conventional database [joins](https://code.kx.com/q/basics/joins/) are primarily associated with tables, in which a join is used to extract data from a lookup table based on a common column or key.

# Simple Table Joins
A join combines data from two tables, or from a table and a dictionary. These subsequent joins are considered to be *simple* joins as they do not require the tables to be modified (e.g. keyed) or structured in a particular way (.e.g time sorted) before  joining. 

## Join rows using  `,`

The [join operator (`,`)](https://code.kx.com/q/ref/join/) is defined for all datatypes and will perform the action of joining the RHS to the end of the LHS input. We can use `,` to append a record to (a copy of) a table, but no type checking will be performed.

In [ ]:
show t:([] name:`Harry`Ron`Hermione; iq:98 42 126)
meta t

In [ ]:
//joining a dictionary to a table (i.e. a list of dictionaries)
show newT:t,`name`iq!(`Dobby; `12)

//joining a table (i.e. a list of dictionaries), to a table (i.e. a list of dictionaries)
t,([]name:enlist`Dobby;iq:enlist `12)

//we see the type of iq is now a mixed list
meta newT

We don't see the same behaviour when the keys do not match: 

In [ ]:
t
t,flip (`name`iq2!(enlist `Bumble;59))  //the columns aren't the same now

Two keyed tables with the same matching columns can be joined with `,` as well. The right operand is upserted into the left.

In [ ]:
show kt:([eid:1001 1002 1003] name:`Harry`Ron`Hermione; iq:98 42 126)
show kt1:([eid:1003 1004] name:`Hermione`Hagrid; iq:`z`)
/kt,(([]eid:1003 1004)!([]name:`Hermione`Hagrid;iq:`z`))   //alternative syntax as dictionary of tables
kt,kt1                                               //upsert

##### Exercise

Use `,` to join the following data:
    
    d:`name`iq!(`Sirius`Ginny;59 42)

 to `t` as: 

1. A single row nested in the table `t`
* Two separate rows in `t`

In [ ]:
show d:`name`iq!(`Sirius`Ginny;59 42)
t

In [ ]:
t,d       //joining one dictionary to a table (i.e. a list of dictionaries) - each displayed as a row

In [ ]:
t,flip d  //swapping our dictionary to a table so each entry is a row before joining

In [ ]:
//your answer here

## Join columns using `,'`

Two tables with the same number of records can be joined pairwise (or "sideways") with join-each `,'` to create a column join in which the columns are aligned in parallel.

In [ ]:
([] c1:`a`b`c),'([] c2:100 200 300)         //join each item of c1 to the correponding item in c2

If we wanted to add a new constant column, we could do the following: 

In [ ]:
//joining our RHS dictionary to each of the dictionaries to the left
([] c1:`a`b`c),\:enlist[`c3]!enlist `constant

This explains why if we use this method we can expect columns to be overwritten:

In [ ]:
([] c1:`a`b`c; c2:1 2 3),'([] c2:100 200 300)    // overwrites common columns 

 ##### Keyed Tables 

A sideways join on keyed tables requires that the key records conform, meaning that the key columns must have identical *meta* - i.e same table columns and column types. 

In [ ]:
([k:1 2 3] v1:10 20 30),'([k:3 4 5] v2:1000 2000 3000)

If these were the same column, the value per our RHS dictionary prevails

In [ ]:
([k:1 2 3] v1:10 20 30),'([k:3 4 5] v1:1000 2000 3000)

Thinking back to Dictionaries - can you remember which operator we can use here to retain the values in our LHS table, if they exist in the LHS and not the RHS? 

In [ ]:
([k:1 2 3] v1:10 20 30)^'([k:3 4 5] v1:1000 2000 3000)  //^ will fill null values in the RHS with the LHS

##### Exercise

Create the following tables:

    t1:([] sym:`a`b`c`d; price:1 2 3 4f)
    t2:([] size:3 4 2 5)
    t3:([] sym: `e`f`a; price: 10 20 30f)

1. Use the join `,'` operator to update the table `t1` to have a `size` column as per `t2`.
2. Use the join `,` operator to combine the `t1` and `t3` tables so the final table has 7 rows. 
3. Apply a key of `sym` to each of t1 and t3. Join these keyed tables so the final table has prices for all syms, and the final value of `a` is 1 i.e. table `t1` values are prioritized.  

In [ ]:
t1:([] sym:`a`b`c`d; price:1 2 3 4f)
t2:([] size:3 4 2 5)
t3:([] sym: `e`f`a; price: 10 20 30f)

In [ ]:
//Question 1  
t1,'t2       //joining t2 as a column to t1

In [ ]:
//Question 2 
t1,t3      //joining t3 as extra rows to t1

In [ ]:
//Question 3 
`sym xkey `t1
`sym xkey `t3
t1 
t3
t3^'t1     //we only want to fill in with t3, we want to keep our values in t1

In [ ]:
//your answer here

# Keyed Joins
Some joins are keyed, meaning that when using two tables the keys in the RHS table will determine the way in which we "match" records within the LHS table.

Here is a list of keyed joins in q:
- [Left Join](https://code.kx.com/v2/ref/lj/) - `lj`
- [Inner Join](https://code.kx.com/v2/ref/ij/) - `ij`

The below joins will be discussed in the practical guidance notebook:
- [Union join](https://code.kx.com/q/ref/uj/) - `uj`  
- [Equi join](https://code.kx.com/q/ref/ej/) - `ej`
- [Plus Join](https://code.kx.com/v2/ref/pj/) - `pj`

## Left Join  `lj`

[Left Join](https://code.kx.com/q/ref/lj/) `lj` is most commonly used to supplement a time-series table with reference data - this appends new columns to our time-series LHS table. The `lj` is one of the most frequently used joins in kdb+/q.

<img src="../LeftJoin.png" width="200" height="200">

The syntax is:

    <time-series table> lj <reference (keyed) table>   

Using the `lj` operator, we can join both these tables together so our trade table is supplemented with the reference information 

In [ ]:
show trade:([]time:09:00+10*til 5;sym:`JPM`GE`JPM`IBM`GE;price:30+5?3.;size:5?20) 

In [ ]:
//created a key table keyed on sym
show reference:([sym:`JPM`IBM`GS]companyName:`$("JP Morgan";"International Business Machines";"Goldman Sachs");sector:`Banking`IT`Banking) 

In [ ]:
trade lj reference   //the common sym column is how we match between the two tables 
                        //reference data is only returned for syms that are present in our reference table

We know that we need the right hand table to be keyed but what happens if we key the `trade` table ?

In [ ]:
(1!trade) lj reference

The key on our LHS table does not affect our join operation, but we see the format of the table returned is determined by the LHS time-series table. 

<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>Be careful of common columns in your two tables! When using a <code>lj</code> kdb+/q assumes that the data in the reference table is what we want to keep, so therefore this is the value we will have in our final table. Be particularly careful with <code>time</code> columns! </i></p>

In [ ]:
show r2:reference,\:enlist[`price]! enlist 3f

In [ ]:
trade lj r2    //we have overwritten our price column!

##### Exercise
Create a second reference table `reference1` consisting of company and [RIC](https://en.wikipedia.org/wiki/Reuters_Instrument_Code) number. The table should be keyed on companyName.

RICS: 
* JP Morgan - 34562
* International Business Machines - 23981

Use this, along with the trade and existing reference table, to create a table that contains the RIC column. 

In [ ]:
show reference1:([companyName:`$("JP Morgan";"International Business Machines")]RIC:(34562;23981))
trade lj reference lj reference1

In [ ]:
//your answer here

## Inner Join `ij`
[Inner join](https://code.kx.com/q/ref/ij/) `ij` operator acts in the same way as the left join but only records for which the key matches between tables will remain in the resultant table. This appends new columns to our time-series LHS table, and removes non-common rows.

<img src="../innerJoin.png" width="200" height="200">

The syntax is: 

    <time-series table> ij <reference (keyed) table>

The difference between the inner join and the left join is that if there is no matching record, it will omit that row in the result. 

When we apply the inner join operator on trade and reference, you can see that the `GE` trades are now omitted from the result:

In [ ]:
trade
reference

In [ ]:
trade lj reference
trade ij reference 

##### Exercise

Using our `trade`, `reference` and `reference1` tables from before: 

1. Create a resultant table that only has the trades for `JPM`, but for those trades has associated companyName, sector and RIC. 

In [ ]:
1 sublist reference1                      //sublisting our reference table to only have JP Morgan RIC 
trade ij reference ij 1 sublist reference1 //inner joining over all 

In [ ]:
//your answer here

2. Create a resultant table that only has the trades for which we have company sector information and only the RIC for `IBM`.

In [ ]:
-1 sublist reference1                           //just IBM record for RIC
(trade ij reference) lj -1 sublist reference1   //ij - only want the trades we have this sector info for
                                                    //lj - we don't want to throw away data we don't have RIC for

In [ ]:
//your answer here

# Bitemporal joins (aka "asof" joins)
Some joins are [bitemporal](https://en.wikipedia.org/wiki/Temporal_database), meaning they take into account the time in two dimensions, or for us, as per two different tables.  

Below is a list of bitemporal joins:
+ asof - combines data based as per the time in one table with the prevailing values in another table. 
+ window-join - combines data based as per the time in one table with the values within a specified window in another table. **Discussed in Practical guidance notebook**

These joins are referred to as "as-of" joins since they are retrieving values in a secondary table, relative to the time as-of another table. 

## Asof Join 

As the name may suggest [Asof join](https://code.kx.com/q/ref/aj/) is mainly used to join columns with reference to time. It will return each row of the source table and the last recorded values per the second table which have an entry at or prior to the time in the source table. The records in the secondary table are filtered based on the specified match columns.
 

The Syntax 

    aj[col1,…,coln,timeCol;source table;reference table]

Where `col1` to `coln` are exact match columns and `timeCol` is the common time column that should be used for the time lookup. 

It might be useful to see an example to better understand this concept, below we will create a quote and trade table:

In [ ]:
show quote:([]time:09:29 09:29 09:32 09:33;sym:`JPM`AAPL`JPM`AAPL;ask:30.23 40.20 30.35 40.35;bid:30.20 40.19 30.33 40.32)

show trade:([]time:09:30 09:31 09:32 09:33 09:34 09:35;sym:`JPM`AAPL`AAPL`JPM`AAPL`JPM;price:30.43 30.45 40.45 30.55 41.00 31.00;size:100 200 200 300 300 600)

Using the asof join, we will be able to determine the prevailing quote for each trade.

In [ ]:
aj[`sym`time;trade;quote]

We can see that only the trades quotes corresponding to the same `sym` where used, and the last previous value as per the quote table was used. 

##### Exercise
Create a table `rack` that has the following times for each symbol in the trade table: 

    times: 09:00 09:30 10:00
    
    
The `rack` table should look like:

|time | sym|
|------|----|
|09:00 |JPM |
|09:00 |AAPL|
|09:30 |JPM |
|09:30 |AAPL|
|10:00 |JPM |
|10:00 |AAPL|

Using this table, find the bid and ask prices as at these times for each particular sym value.

In [ ]:
show rack:([]time:09:00 09:30 10:00) cross ([]sym:`JPM`AAPL)  //people create "racks" for times and syms of interest!
/rack:([]sym:`JPM`AAPL) cross ([]time:09:00 09:30 10:00) //also fine
aj[`sym`time;rack;quote]

In [ ]:
//Your answer here